# Group Lasso with Prior Center

This notebook simulates grouped data and compares the vanilla group lasso (penalizing toward zero) with the new prior-centered version (penalizing toward a provided coefficient vector).

In [1]:
import sys, pathlib

# Add project root (directory containing ``skglm``) to sys.path without installing
def _find_repo_root(start: pathlib.Path):
    for p in [start] + list(start.parents):
        if (p / "skglm").is_dir():
            return p
    return None

here = pathlib.Path.cwd().resolve()
repo_root = _find_repo_root(here)
if repo_root is None:
    raise RuntimeError("Could not find repository root containing 'skglm' directory")
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

import numpy as np
from numpy.linalg import norm

from skglm.estimators import GroupLasso
from skglm.utils.data import grp_converter

rng = np.random.default_rng(0)

## Simulate grouped data
- Features split into equal-sized groups.
- True coefficients are drawn around a non-zero prior.
- Noise is added to the response.

In [2]:
n_samples = 300
n_features = 20
group_size = 4  # contiguous groups of equal size
sigma_noise = 0.5

# Build groups (contiguous blocks)
grp_indices, grp_ptr = grp_converter(group_size, n_features)

# Prior centered away from zero on first two groups
prior = np.zeros(n_features)
prior[: group_size] = 1.5
prior[group_size: 2 * group_size] = -1.0

# True coefficients deviate slightly from the prior
beta_true = prior.copy()
beta_true[: 2 * group_size] += rng.normal(0, 0.25, size=2 * group_size)

# Design and response
X = rng.normal(size=(n_samples, n_features))
y = X @ beta_true + rng.normal(scale=sigma_noise, size=n_samples)

## Fit vanilla vs. prior-centered Group Lasso
- Same regularization strength and group weights.
- `fit_intercept=False` so the prior only targets coefficients.

In [3]:
alpha = 0.2
weights = np.ones(len(grp_ptr) - 1)

model_vanilla = GroupLasso(
    groups=group_size,
    alpha=alpha,
    weights=weights,
    fit_intercept=False,
    max_iter=50,
    max_epochs=5000,
    tol=1e-6,
    verbose=0,
)
model_prior = GroupLasso(
    groups=group_size,
    alpha=alpha,
    weights=weights,
    fit_intercept=False,
    max_iter=50,
    max_epochs=5000,
    tol=1e-6,
    verbose=0,
    prior=prior,
)

model_vanilla.fit(X, y)
model_prior.fit(X, y)

w_vanilla = model_vanilla.coef_
w_prior = model_prior.coef_

## Compare solutions
- Distance to prior and to ground truth.
- Group-wise norms to see shrinkage behavior.

In [4]:
def group_norms(w, grp_ptr):
    norms = []
    for g in range(len(grp_ptr) - 1):
        sl = slice(grp_ptr[g], grp_ptr[g + 1])
        norms.append(norm(w[sl]))
    return np.array(norms)

print(f"||w_vanilla - prior||_2 = {norm(w_vanilla - prior):.4f}")
print(f"||w_prior   - prior||_2 = {norm(w_prior - prior):.4f}")
print(f"||w_vanilla - beta_true||_2 = {norm(w_vanilla - beta_true):.4f}")
print(f"||w_prior   - beta_true||_2 = {norm(w_prior - beta_true):.4f}\n")

print("Group norms (vanilla):", group_norms(w_vanilla, grp_ptr))
print("Group norms (prior)  :", group_norms(w_prior, grp_ptr))
print("Group norms (prior vector):", group_norms(prior, grp_ptr))

||w_vanilla - prior||_2 = 0.6503
||w_prior   - prior||_2 = 0.3115
||w_vanilla - beta_true||_2 = 0.3385
||w_prior   - beta_true||_2 = 0.2032

Group norms (vanilla): [2.88783825 1.51512908 0.         0.         0.        ]
Group norms (prior)  : [3.00528036 1.81709399 0.         0.         0.        ]
Group norms (prior vector): [3. 2. 0. 0. 0.]


You should see the prior-centered solution pulled closer to the supplied prior on the first two groups while leaving uninformative groups shrunk toward zero. Adjust `alpha`, `sigma_noise`, or the group sizes to explore different regimes.